In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
data_matches = pd.read_csv("../../data/football_matches_a.csv", dtype={'manager_pt1': str, 'manager_pt2': str})
data_matches_test_case = data_matches.copy(deep=True)
data_matches['date'] = pd.to_datetime(data_matches['date'], dayfirst=True)
data_matches['date'] = data_matches['date'].dt.date
data_matches.sort_values(by='date', inplace=True)
data_matches.reset_index(inplace=True, drop=True)

/var/folders/bl/xbg_536n2_n123s38z4hxbx00000gn/T/ipykernel_10433/633912701.py:1: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data_matches = pd.read_csv("../../data/football_matches_a.csv", dtype={'manager_pt1': str, 'manager_pt2': str})


In [3]:
assert len(data_matches) == len(data_matches_test_case), "Length of new df not equal to original."

In [4]:
def duplicate_to_team_and_opponent(df_matches):
    df_matches = df_matches[['match_id', 'league', 'date', 'pt1', 'pt2', 'score_pt1', 'score_pt2']]
    df_matches['conceded_pt1'] = df_matches['score_pt2']
    df_matches['conceded_pt2'] = df_matches['score_pt1']

    df_matches_copy = df_matches.copy()
    df_matches = df_matches.rename(columns={'pt1': 'team', 'pt2': 'opponent', 'score_pt1': 'team_goals_scored',
                                            'score_pt2': 'opponent_goals_scored', 'conceded_pt1': 'team_goals_conceded',
                                            'conceded_pt2': 'opponent_goals_conceded'})
    df_matches_copy = df_matches_copy.rename(columns={'pt2': 'team', 'pt1': 'opponent', 'score_pt2': 'team_goals_scored',
                                            'score_pt1': 'opponent_goals_scored', 'conceded_pt2': 'team_goals_conceded',
                                            'conceded_pt1': 'opponent_goals_conceded'})
    # change the below, to be the columns that are in the initial df plus the new ones created.
    df_matches_copy = df_matches_copy[['match_id', 'league', 'date', 'team', 'opponent', 'team_goals_scored', 'opponent_goals_scored',
                                        'team_goals_conceded', 'opponent_goals_conceded',
                                        ]]
    df_matches.loc[:, 'home'] = 1
    df_matches_copy.loc[:, 'home'] = 0
    df_matches = pd.concat([df_matches, df_matches_copy])
    df_matches.sort_values(by='date', inplace=True)
    df_matches.reset_index(inplace=True, drop=True)

    return df_matches

In [5]:
data_matches = duplicate_to_team_and_opponent(data_matches)

In [6]:
assert len(data_matches) == len(data_matches_test_case)*2, "Length of duplicated df not equal to original."

In [7]:
data_matches.head()

,match_id,league,date,team,opponent,team_goals_scored,opponent_goals_scored,team_goals_conceded,opponent_goals_conceded,home
0,l_r_vicenza_sampdoria_1997-08-31,Serie A,1997-08-31,sampdoria,l_r_vicenza,2.0,1.0,1.0,2.0,1
1,empoli_roma_1997-08-31,Serie A,1997-08-31,roma,empoli,3.0,1.0,1.0,3.0,0
2,atalanta_bologna_1997-08-31,Serie A,1997-08-31,bologna,atalanta,2.0,4.0,4.0,2.0,0
3,juventus_lecce_1997-08-31,Serie A,1997-08-31,lecce,juventus,0.0,2.0,2.0,0.0,0
4,l_r_vicenza_sampdoria_1997-08-31,Serie A,1997-08-31,l_r_vicenza,sampdoria,1.0,2.0,2.0,1.0,0


In [8]:
def split_home_and_away(df):
    df_home = df[df['home']==1].copy(deep=True)
    df_away = df[df['home']==0].copy(deep=True)
    return df_home, df_away

def calculate_rolling_mean_team(df, df_home, df_away, stat_list):
    for stat in stat_list:
        df.loc[:, stat + '_avg'] = df[stat].shift(1).rolling(19).mean()
        if not any(substring in stat for substring in ['home', 'away']):
            df_home.loc[:, stat + '_avg_home'] = df_home[stat].shift(1).rolling(19).mean()
            df_away.loc[:, stat + '_avg_away'] = df_away[stat].shift(1).rolling(19).mean()
    
    df_stat_home_and_away = pd.concat([df_home, df_away])
    avg_cols = [col for col in df_stat_home_and_away.columns if 'avg' in col]
    df_stat_home_and_away = df_stat_home_and_away[avg_cols + ['match_id', 'team']]
    df_merge = df.merge(df_stat_home_and_away, on=['match_id', 'team'], how='inner',
#              validate='1:1'
            )
    return df_merge

def get_avg_columns(df):
    return [col for col in df.columns if 'avg' in col]

def calc_rolling_average(df, team, stat_list, for_team=True):
    if for_team:
        df_team = df[df['team']==team]
    else:
        df_team = df[df['opponent']==team]
    df_home, df_away = split_home_and_away(df_team)
    df_merge = calculate_rolling_mean_team(df_team, df_home, df_away, stat_list)
    return df_merge
    
def get_rolling_average(df, stats_list, for_team=True):
    teams_list = df['team'].unique().tolist()
    df['row_num'] = np.arange(1, len(df)+1)
    df_average = pd.DataFrame(columns=df.columns)

    for team in tqdm(teams_list):
        df_temp = calc_rolling_average(df, team, stats_list, for_team)
        df_average = pd.concat([df_average, df_temp])

    df_average.drop('row_num', axis=1, inplace=True)

    return df_average

In [9]:
# test
df_test = data_matches.copy(deep=True)
df_home_test, df_away_test = split_home_and_away(df_test)

In [10]:
assert len(df_home_test) == len(data_matches_test_case), 'home matches missing.'
assert len(df_away_test) == len(data_matches_test_case), 'away matches missing.'

In [11]:
team_stats = ['team_goals_scored',
              'team_goals_conceded',
              ]
opponent_stats = ['opponent_goals_scored',
                  'opponent_goals_conceded',
                  ]

In [12]:
data_averages_team = get_rolling_average(df_test, team_stats)
data_averages_opponent = get_rolling_average(df_test, opponent_stats, for_team=False)

100%|█████████████████████████████████████████| 681/681 [00:07<00:00, 93.53it/s]


In [15]:
def pivot_df(df, scored_or_conceded):
    return df.pivot_table('team_goals_' + scored_or_conceded, index='date', columns='league')

def check_columns(df_true, df_test):
    assert df_true.columns.tolist().sort() == df_test.columns.to_list().sort(), "Column for home and away not identical."
    
def calculate_averages(df, column, home_away):
    df_copy = df.copy().reset_index()
    s_h = df_copy[['date', column]].copy()
    s_h[column + f'_avg_{home_away}'] = s_h[column].dropna().shift(1).rolling(19).mean()
    s_h.drop(column, axis=1, inplace=True)
    df_avg = df.merge(s_h, left_on='date', right_on='date')
    return df_avg

def merge_on_common_columns(df1, df2):
    common_columns = list(set(df1.columns).intersection(df2.columns))
    df = pd.merge(df1, df2, on=common_columns)
    return df

In [16]:
def get_league_average(df, scored_or_conceded):
    df_home, df_away = split_home_and_away(df)
    df_league_home_avg = pivot_df(df_home, scored_or_conceded)
    df_league_away_avg = pivot_df(df_away, scored_or_conceded)
    check_columns(df_league_home_avg, df_league_away_avg)
    columns = df_league_home_avg.columns
    
    df_league_home_avg_test = df_league_home_avg.copy(deep=True)
    df_league_away_avg_test = df_league_away_avg.copy(deep=True)
    
    for column in columns:
        df_league_home_avg = calculate_averages(df_league_home_avg, column, 'home')
        df_league_away_avg = calculate_averages(df_league_away_avg, column, 'away')
        
    assert df_league_home_avg.shape == df_league_away_avg.shape, "Home and away are different shapes"

    df_league_home_avg = df_league_home_avg.ffill()
    df_league_away_avg = df_league_away_avg.ffill()

    df_league_home_avg.reset_index(inplace=True)
    df_league_away_avg.reset_index(inplace=True)

    df_home_avg_merge = pd.merge(df, df_league_home_avg.melt(id_vars='date')
                                    # .assign(league = lambda x: x['variable'])
                                    , on=['league','date'])
    df_away_avg_merge = pd.merge(df, df_league_away_avg.melt(id_vars='date')
                                    # .assign(league = lambda x: x['variable'])
                                    , on=['league','date'])

    df_home_avg_merge.rename(columns={'value': 'league_home_goals_'  + scored_or_conceded}, inplace=True)
    df_away_avg_merge.rename(columns={'value': 'league_away_goals_'  + scored_or_conceded}, inplace=True)

    df_merge = df_home_avg_merge.copy()
    df_merge['league_away_goals_' + scored_or_conceded] = df_away_avg_merge['league_away_goals_' + scored_or_conceded]

    columns = columns.values.tolist()
    df_league_home_avg.drop(columns, axis=1, inplace=True)
    df_league_away_avg.drop(columns, axis=1, inplace=True)
    for column in df_league_home_avg.columns:
        if column[-9:] == '_avg_home':
            df_league_home_avg.rename(columns={column: column[:-9]}, inplace=True)
    for column in df_league_away_avg.columns:
        if column[-9:] == '_avg_away':
            df_league_away_avg.rename(columns={column: column[:-9]}, inplace=True)
    
    df_home_avg_merge = pd.merge(df_home_avg_merge, df_league_home_avg.melt(id_vars='date')
                                    # .assign(league = lambda x: x['variable'])
                                    , on=['league','date'])
    df_away_avg_merge = pd.merge(df_away_avg_merge, df_league_away_avg.melt(id_vars='date')
                                    # .assign(league = lambda x: x['variable'])
                                    , on=['league','date'])

    df_home_avg_merge.rename(columns={'value': 'league_home_goals_'  + scored_or_conceded + '_avg'}, inplace=True)
    df_away_avg_merge.rename(columns={'value': 'league_away_goals_'  + scored_or_conceded + '_avg'}, inplace=True)

    df_merge_avg = df_home_avg_merge.copy()
    df_merge_avg['league_away_goals_' + scored_or_conceded + '_avg'] = df_away_avg_merge['league_away_goals_' + scored_or_conceded + '_avg']
    df_merge = merge_on_common_columns(df_merge, df_merge_avg)
    df_merge.sort_values(by='date', inplace=True)

    return df_merge

In [17]:
data_averages = merge_on_common_columns(data_averages_team, data_averages_opponent)
assert len(data_averages) == len(data_matches_test_case)*2, 'matches missing'

In [18]:
df_test = get_league_average(df_test, 'scored')
df_test = get_league_average(df_test, 'conceded')

In [19]:
assert len(df_test) == len(data_matches_test_case)*2, "League averages are wrong length."

In [20]:
data_averages = merge_on_common_columns(data_averages, df_test)

In [21]:
assert len(data_averages) == len(data_matches_test_case)*2, "League averages are wrong length."

In [22]:
def calc_strength(df):
    conditions = [
        df['home'] == 1,
        df['home'] == 0
    ]

    output_team_score = [
        df['team_goals_scored_avg_home'] / df['league_home_goals_scored_avg'],
        df['team_goals_scored_avg_away'] / df['league_away_goals_scored_avg']
    ]

    output_team_concede = [
        df['team_goals_conceded_avg_home'] / df['league_home_goals_conceded_avg'],
        df['team_goals_conceded_avg_away'] / df['league_away_goals_conceded_avg']
    ]

    output_opponent_score = [
        df['opponent_goals_scored_avg_away'] / df['league_away_goals_scored_avg'],
        df['opponent_goals_scored_avg_home'] / df['league_home_goals_scored_avg']
    ]

    output_opponent_concede = [
        df['opponent_goals_conceded_avg_away'] / df['league_away_goals_conceded_avg'],
        df['opponent_goals_conceded_avg_home'] / df['league_home_goals_conceded_avg']
    ]

    df['team_attack_strength'] = np.select(conditions, output_team_score, 'Other').astype(float)
    df['team_defense_strength'] = np.select(conditions, output_team_concede, 'Other').astype(float)
    df['opponent_attack_strength'] = np.select(conditions, output_opponent_score, 'Other').astype(float)
    df['opponent_defense_strength'] = np.select(conditions, output_opponent_concede, 'Other').astype(float)

    df.replace(np.nan, 0, inplace=True)
    df.replace('nan', 0, inplace=True)

    return df

In [23]:
def calc_lambda(df):
    strength_columns = ['team_attack_strength',
                        'team_defense_strength',
                        'opponent_attack_strength',
                        'opponent_defense_strength']
    df[strength_columns].fillna(0, inplace=True)
    conditions = [
        df['home'] == 1,
        df['home'] == 0,
    ]

    output_team = [
        df['league_home_goals_scored_avg'] * df['team_attack_strength'] * df['opponent_defense_strength'],
        df['league_away_goals_scored_avg'] * df['team_attack_strength'] * df['opponent_defense_strength'],
    ]

    output_opponent = [
        df['league_away_goals_scored_avg'] * df['opponent_attack_strength'] * df['team_defense_strength'],
        df['league_home_goals_scored_avg'] * df['opponent_attack_strength'] * df['team_defense_strength'],
    ]

    df['team_lambda'] = np.select(conditions, output_team, 'Other')
    df['opponent_lambda'] = np.select(conditions, output_opponent, 'Other')

    return df

In [24]:
data_averages = calc_strength(data_averages)
data_averages = calc_lambda(data_averages)
data_averages.reset_index(inplace=True, drop=True)

In [25]:
assert len(data_averages) == len(data_matches_test_case)*2, "Different to imported data length."

## Checking final data set

We will test this by checking that for two rows that correspond to the same match, the contain the same data, just flipped.

In [28]:
data = pd.read_csv("../../data/test_wavenet.csv", index_col=0)

In [29]:
data.head()

,league,date,team,opponent,result,result_0,elo_team,elo_opponent,elo_diff,home,...,league_home_goals_conceded_7_y,league_away_goals_conceded_7_y,league_home_goals_conceded_avg_7_y,league_away_goals_conceded_avg_7_y,team_attack_strength_7_y,team_defense_strength_7_y,opponent_attack_strength_7_y,opponent_defense_strength_7_y,team_lambda_7_y,opponent_lambda_7_y
0,Serie B,2012-09-15,a_s_d_lanciano_calcio_1920,cittadella,0.0,0,1439.227712,1432.282433,6.945279,0.0,...,1.615385,1.846154,1.395872,1.438273,0.841652,1.018040,0.791809,0.768465,0.930247,1.125202
1,Serie B,2012-09-22,a_s_d_lanciano_calcio_1920,grosseto,0.5,0,1418.827963,1406.751371,12.076591,1.0,...,1.142857,1.500000,1.419488,1.482808,0.778635,0.816374,0.709891,0.964024,1.065500,0.859341
2,Serie B,2012-09-25,a_s_d_lanciano_calcio_1920,reggina,1.0,0,1418.133058,1437.187196,-19.054137,0.0,...,1.125000,1.625000,1.503801,2.283041,0.622439,0.979973,0.559984,0.714652,1.015558,0.825240
3,Serie B,2012-09-29,a_s_d_lanciano_calcio_1920,modena,0.5,0,1439.228804,1501.037840,-61.809036,1.0,...,1.000000,1.000000,1.190165,1.545966,0.987290,1.149774,1.149774,1.021334,1.558878,1.573375
4,Serie B,2012-10-06,a_s_d_lanciano_calcio_1920,brescia,0.0,0,1442.749752,1444.233023,-1.483271,0.0,...,1.250000,1.583333,1.107230,1.501840,0.630805,1.378499,0.950689,0.700894,0.664005,1.451051


In [30]:
data[data['team']=='juventus']

,league,date,team,opponent,result,result_0,elo_team,elo_opponent,elo_diff,home,...,league_home_goals_conceded_7_y,league_away_goals_conceded_7_y,league_home_goals_conceded_avg_7_y,league_away_goals_conceded_avg_7_y,team_attack_strength_7_y,team_defense_strength_7_y,opponent_attack_strength_7_y,opponent_defense_strength_7_y,team_lambda_7_y,opponent_lambda_7_y
30141,Serie A,1997-11-09,juventus,napoli,1.0,0,1575.166302,1441.459893,133.706409,0.0,...,1.444444,1.777778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30142,Serie A,1997-11-23,juventus,parma,0.5,0,1584.662778,1563.492946,21.169833,1.0,...,1.666667,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30143,Serie A,1997-11-30,juventus,milan,0.5,0,1583.749932,1499.816123,83.933809,0.0,...,1.111111,1.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30144,Serie A,1997-12-06,juventus,lazio,1.0,0,1580.195106,1512.629440,67.565666,1.0,...,2.000000,2.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30145,Serie A,1997-12-14,juventus,piacenza,0.5,0,1592.314286,1457.030985,135.283302,0.0,...,1.000000,1.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31270,Serie A,2023-01-13,juventus,napoli,0.0,0,1654.291836,1788.069537,-133.777701,0.0,...,1.111111,1.000000,1.407126,1.440624,1.421337,0.584542,0.986414,0.635861,1.271723,0.830665
31271,Coppa Italia,2023-01-19,juventus,monza,1.0,0,1629.475412,1544.091466,85.383947,1.0,...,2.250000,0.750000,1.336658,1.485361,1.204740,0.590633,0.630008,0.992139,1.775406,0.497375
31272,Serie A,2023-01-22,juventus,atalanta,0.5,0,1627.580126,1604.992337,22.587789,1.0,...,1.200000,1.500000,1.379202,1.370449,0.998520,0.915862,1.411953,0.537665,0.735752,1.783520
31273,Serie A,2023-01-29,juventus,monza,0.0,0,1626.606305,1529.103629,97.502676,1.0,...,0.600000,1.400000,1.207310,1.379240,1.351417,1.373755,1.221115,0.915476,1.493671,2.313692


In [31]:
test_match = {
    'date': '2023-02-02',
    'team': 'juventus',
    'opp': 'lazio'
}

In [32]:
def get_match(df, date, team, opp):
    return df[(df['date']==date) & ((df['team']==team) | (df['team']==opp))]

In [40]:
def test_team_opp_stats(df, date, team, opp):
    match_df = get_match(df, date, team, opp)
    return match_df

In [41]:
test_case = test_team_opp_stats(data, test_match['date'], test_match['team'], test_match['opp'])
test_case.head()

,league,date,team,opponent,result,result_0,elo_team,elo_opponent,elo_diff,home,...,league_home_goals_conceded_7_y,league_away_goals_conceded_7_y,league_home_goals_conceded_avg_7_y,league_away_goals_conceded_avg_7_y,team_attack_strength_7_y,team_defense_strength_7_y,opponent_attack_strength_7_y,opponent_defense_strength_7_y,team_lambda_7_y,opponent_lambda_7_y
31274,Coppa Italia,2023-02-02,juventus,lazio,1.0,0,1596.560386,1632.589353,-36.028967,1.0,...,1.083333,1.083333,1.188764,1.209190,1.461049,1.088158,1.131684,0.752662,1.307255,1.489058
33849,Coppa Italia,2023-02-02,lazio,juventus,0.0,0,1632.589353,1596.560386,36.028967,0.0,...,1.214286,1.785714,1.212573,1.264327,1.456984,0.651073,1.475766,0.999075,1.840401,1.165078


In [43]:
test_case.to_csv("../../data/test_wavenet_match.csv")

This has been checked in excel that the past histories for each team are correct, and just flipped.